In [1]:
import pandas as pd
import numpy as np
import pprint as pp # for printing
import scipy.stats as st # for Normal PDF
# Plotting libraries 
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
import itertools
import warnings
warnings.filterwarnings("ignore")# Silence warnings 
plt.style.use('ggplot')

In [2]:
# Set seed
np.random.seed(1234)

# read in data
turnout = pd.read_csv("turnout.csv")

# Train-Test split (just using Pandas)
train = turnout.sample(frac=0.8).reset_index(drop=True)
test = turnout.drop(train.index).reset_index(drop=True)

# Print off the split count 
print("Training Data:",train.shape[0],
      "\nTest Data:",test.shape[0])

# Look at the head of the data
train.head()

Training Data: 1600 
Test Data: 400


,id,age,educate,income,vote,white
0,1749,78,16.0,1.3131,1,1
1,935,72,3.0,0.6765,1,0
2,1034,64,10.0,1.3131,1,1
3,244,80,8.0,1.1839,1,1
4,929,19,14.0,2.9072,1,1


### Calculate Class Probabilities: Pr(Class)

In [3]:
N = train.shape[0]

# Subset the data by class
vote1 = train.query("vote == 1")
vote0 = train.query("vote == 0")

# Calculate the probability for each class
pr_vote_1 = vote1.shape[0]/N
pr_vote_0 = vote0.shape[0]/N

# Print the probabilities
print(
f"""
Pr(vote = 1): {pr_vote_1}
Pr(vote = 0): {pr_vote_0}
""")


Pr(vote = 1): 0.7425
Pr(vote = 0): 0.2575



### Calculate the Conditional Probabilities Pr(data|class)

In [4]:
# Given vote == 1
w1_v1 = vote1.query("white == 1").shape[0]/vote1.shape[0]
w0_v1 = vote1.query("white == 0").shape[0]/vote1.shape[0]


# Given vote == 0
w1_v0 = vote0.query("white == 1").shape[0]/vote0.shape[0]
w0_v0 = vote0.query("white == 0").shape[0]/vote0.shape[0]


print(
f"""
Pr(white = 1 |vote = 1): {w1_v1}
Pr(white = 0 |vote = 1): {w0_v1}
Pr(white = 1 |vote = 0): {w1_v0}
Pr(white = 0 |vote = 0): {w0_v0}
""")


Pr(white = 1 |vote = 1): 0.8686868686868687
Pr(white = 0 |vote = 1): 0.13131313131313133
Pr(white = 1 |vote = 0): 0.7985436893203883
Pr(white = 0 |vote = 0): 0.20145631067961164



### Calculate the conditional means/standard deviations

In [5]:
# Collect the mean and standard dev. of each conditional distribution
dist_locs = \
{("age",1):{'mean':vote1.age.mean(),'sd':vote1.age.std()},
 ("age",0):{'mean':vote0.age.mean(),'sd':vote0.age.std()},
 ("educate",1):{'mean':vote1.educate.mean(),'sd':vote1.educate.std()},
 ("educate",0):{'mean':vote0.educate.mean(),'sd':vote0.educate.std()},
 ("income",1):{'mean':vote1.income.mean(),'sd':vote1.income.std()},
 ("income",0):{'mean':vote0.income.mean(),'sd':vote0.income.std()}
}

# Print
pp.pprint(dist_locs)

{('age', 0): {'mean': 42.601941747572816, 'sd': 19.147825402160812},
 ('age', 1): {'mean': 46.32491582491583, 'sd': 16.924844588853716},
 ('educate', 0): {'mean': 10.62864077669903, 'sd': 3.304381091983527},
 ('educate', 1): {'mean': 12.558922558922559, 'sd': 3.295714127444309},
 ('income', 0): {'mean': 2.7381618932038836, 'sd': 2.2429913729337625},
 ('income', 1): {'mean': 4.229461952861947, 'sd': 2.8482089910676964}}


### Making predictions using test data

In [6]:
x1,x2,x3,x4 = test.loc[4,['age','educate','income','white']]

In [7]:
# Prediction for the 1 class
a1 = st.norm(dist_locs[("age",1)]['mean'], dist_locs[("age",1)]['sd']).pdf(x1)
b1 = st.norm(dist_locs[("educate",1)]['mean'], dist_locs[("educate",1)]['sd']).pdf(x2)
c1 = st.norm(dist_locs[("income",1)]['mean'], dist_locs[("income",1)]['sd']).pdf(x3)
d1 = pr_vote_1
if x4: e1 = w1_v1
else: e1 = w0_v1

pr_1 = a1 * b1 * c1 * d1 * e1

# Prediction for the 0 class
a0 = st.norm(dist_locs[("age",0)]['mean'], dist_locs[("age",0)]['sd']).pdf(x1)
b0 = st.norm(dist_locs[("educate",0)]['mean'], dist_locs[("educate",0)]['sd']).pdf(x2)
c0 = st.norm(dist_locs[("income",0)]['mean'], dist_locs[("income",0)]['sd']).pdf(x3)
d0 = pr_vote_0
if x4: e0 = w1_v0
else: e0 = w0_v0 

pr_0 = a0 * b0 * c0 * d0 * e0

print(
f'''
    Pr(y == 1| X): {pr_1}
    Pr(y == 0| X): {pr_0}
''')


    Pr(y == 1| X): 4.0026272936715817e-05
    Pr(y == 0| X): 1.7289840025929505e-06



In [8]:
store_preds = []
for i,row in test.iterrows():
    _,x1,x2,x3,_,x4 = row.values
    # Prediction for the 1 class
    a1 = st.norm(dist_locs[("age",1)]['mean'], dist_locs[("age",1)]['sd']).pdf(x1)
    b1 = st.norm(dist_locs[("educate",1)]['mean'], dist_locs[("educate",1)]['sd']).pdf(x2)
    c1 = st.norm(dist_locs[("income",1)]['mean'], dist_locs[("income",1)]['sd']).pdf(x3)
    d1 = pr_vote_1
    if x4: e1 = w1_v1
    else: e1 = w0_v1

    pr_1 = a1 * b1 * c1 * d1 * e1

    # Prediction for the 0 class
    a0 = st.norm(dist_locs[("age",0)]['mean'], dist_locs[("age",0)]['sd']).pdf(x1)
    b0 = st.norm(dist_locs[("educate",0)]['mean'], dist_locs[("educate",0)]['sd']).pdf(x2)
    c0 = st.norm(dist_locs[("income",0)]['mean'], dist_locs[("income",0)]['sd']).pdf(x3)
    d0 = pr_vote_0
    if x4: e0 = w1_v0
    else: e0 = w0_v0 

    pr_0 = a0 * b0 * c0 * d0 * e0
    
    store_preds.append([pr_0,pr_1,max([(pr_0,0),(pr_1,1)])[1]])
prediction = pd.DataFrame(store_preds,columns=["pr_0","pr_1","pred"])

prediction

,pr_0,pr_1,pred
0,0.000004,0.000015,1
1,0.000020,0.000179,1
2,0.000030,0.000068,1
3,0.000073,0.000187,1
4,0.000002,0.000040,1
...,...,...,...
395,0.000016,0.000070,1
396,0.000076,0.000175,1
397,0.000002,0.000063,1
398,0.000050,0.000056,1


In [9]:
accuracy = sum(test.vote == prediction.pred)/test.shape[0]
accuracy

0.715

#### Conclusion

We obtained predictive accuracy of 71.5% on the training data. Although it's better than a coin flip (50%), it's not as good as just saying everyone will vote (74.25%)